# Challenge SD207 - 2017
*<p>Author: Pengfei MI, Rui SONG</p>*
*<p>Date: 06/06/2017</p>*

In [1]:
# Basic libraries
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import platform
from time import time

# Librosa related: audio feature extraction
import librosa
import librosa.display

# Sklearn related: data preprocessing and classifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.base import clone

In [2]:
# Define some usefull functions
def load_sound_file(file_name):
    X, sr = librosa.load(os.path.join(FILEROOT, file_name), sr=None)
    return X

def extract_feature(file_name, feature_type): # Late fusion
    X, sample_rate = librosa.load(os.path.join(FILEROOT, file_name), sr=None)
    if feature_type == 'mfcc':
        return librosa.feature.mfcc(y=X, sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc).T
    elif feature_type == 'mfcc_0':
        mfcc = librosa.feature.mfcc(y=X, sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc)[1:].T
        return mfcc
    elif feature_type == "d_mfcc":
        mfcc = librosa.feature.mfcc(y=X, sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc).T
        return librosa.feature.delta(mfcc, width=width, order=1, trim=True)
    elif feature_type == "dd_mfcc":
        mfcc = librosa.feature.mfcc(y=X, sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc).T
        return librosa.feature.delta(mfcc, width=width, order=2, trim=True)
    elif feature_type == "mfcc_d":
        mfcc = librosa.feature.mfcc(y=X, sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc).T
        d_mfcc = librosa.feature.delta(mfcc, width=width, order=1, trim=True)
        return np.c_[mfcc, d_mfcc]
    elif feature_type == "mfcc_dd":
        mfcc = librosa.feature.mfcc(y=X, sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=n_mfcc).T
        d_mfcc = librosa.feature.delta(mfcc, width=width, order=1, trim=True)
        dd_mfcc = librosa.feature.delta(mfcc, width=width, order=2, trim=True)
        return np.c_[mfcc, d_mfcc, dd_mfcc]
    
def parse_audio_files(file_names, file_labels, feature_type):
    # Try to detect the feature dimension
    n_features = extract_feature(file_names[0], feature_type).shape[1]
    features, labels = np.empty((0, n_features)), np.empty(0)
    for fn, fl in zip(file_names, file_labels):
        ff = extract_feature(fn, feature_type)
        features = np.vstack([features, ff])
        labels = np.append(labels, fl*np.ones(ff.shape[0]))
    return np.array(features), np.array(labels, dtype = np.int)

def cross_validation(clf, X, y, test_fold, feature_type="mfcc"):
    y_pred, y_pred_sum, y_pred_prod = np.empty_like(y), np.empty_like(y), np.empty_like(y)
    n_folds = len(np.unique(test_fold))
    for i in range(n_folds):
        t0 = time()
        new_clf = clone(clf, safe=True)
        X_train = X[test_fold != i]
        X_test = X[test_fold == i]
        y_train = y[test_fold != i]
        y_test = y[test_fold == i]
        print "Launching fold #%d/%d, train set size: %d, test set size: %d" % (i+1, n_folds, len(X_train), len(X_test))
        clf_train(new_clf, X_train, y_train, feature_type)
        test_pred, test_pred_sum, test_pred_prod = clf_predict(new_clf, X_test, feature_type)
        y_pred[test_fold == i] = test_pred
        y_pred_sum[test_fold == i] = test_pred_sum
        y_pred_prod[test_fold == i] = test_pred_prod
        print "fold#%d done in %0.3fs, score: %0.3f." % (i+1, time()-t0, accuracy_score(y_test, test_pred))
    t0 = time()
    print "Retraining classifier with whole train set..."
    clf_train(clf, X, y, feature_type)
    print "Done in %0.3fs." % (time() - t0)
    return y_pred, y_pred_sum, y_pred_prod

def clf_train(clf, files, file_labels, feature_type):
    X_train, y_train= parse_audio_files(files, file_labels, feature_type)
    clf.fit(X_train, y_train)
        
def predict_maj(clf, X_test, feature_type):
    y_pred = np.empty(0)
    for x in X_test:
        x_mfccs = extract_feature(x, feature_type)
        y_predicts = clf.predict(x_mfccs)
        y_pred = np.append(y_pred, sp.stats.mode(y_predicts).mode[0])
    return np.array(y_pred, dtype = np.int)

def predict_sum(clf, X_test, feature_type):
    y_pred = np.empty(0)
    for x in X_test:
        x_mfccs = extract_feature(x, feature_type)
        y_predicts = np.sum(clf.predict_proba(x_mfccs), axis=0)
        y_pred = np.append(y_pred, np.argmax(y_predicts))
    return np.array(y_pred, dtype = np.int)

def predict_prod(clf, X_test, feature_type):
    y_pred = np.empty(0)
    for x in X_test:
        x_mfccs = extract_feature(x, feature_type)
        y_predicts = np.prod(clf.predict_proba(x_mfccs), axis=0)
        y_pred = np.append(y_pred, np.argmax(y_predicts))
    return np.array(y_pred, dtype = np.int)

def clf_predict(clf, X_test, feature_type):
    y_pred = np.empty(0)
    y_pred_sum = np.empty(0)
    y_pred_prod = np.empty(0)
    for x in X_test:
        x_mfccs = extract_feature(x, feature_type)
        y_predicts = clf.predict(x_mfccs)
        y_predict_probas = clf.predict_proba(x_mfccs)
        y_pred = np.append(y_pred, sp.stats.mode(y_predicts).mode[0])
        y_pred_sum = np.append(y_pred_sum, np.argmax(np.sum(y_predict_probas, axis=0)))
        y_pred_prod = np.append(y_pred_prod, np.argmax(np.prod(y_predict_probas, axis=0)))
    return np.array(y_pred, dtype=np.int), np.array(y_pred_sum, dtype=np.int), np.array(y_pred_prod, dtype=np.int)

def predict_calibration(clfs, feature_types, pcss, X_test):
    y_pred, y_pred_sum = np.empty(0), np.empty(0)
    for x in X_test:
        max_pcs, max_idx = -1,-1
        proba_sum = np.empty(n_labels)
        for (i, clf) in enumerate(clfs):
            x_feature = extract_feature(x, feature_types[i])
            x_pred_proba = np.mean(clf.predict_proba(x_feature), axis=0)
            proba_sum += x_pred_proba*pcss[i]
            res = np.argmax(x_pred_proba)
            if pcss[i][res] > max_pcs:
                max_pcs = pcss[i][res]
                max_idx = res
        y_pred = np.append(y_pred, max_idx)
        y_pred_sum = np.append(y_pred_sum, np.argmax(proba_sum))
    return np.array(y_pred, dtype=np.int), np.array(y_pred_sum, dtype=np.int)

In [3]:
# Read data and preprocessing
print "Loading files..."
t0 = time()

# Define FILEROOT according to the platform
# My personal computer
if sys.platform == "darwin":
    FILEROOT = './'
# Node of Telecom
elif platform.node()[:4] == 'lame':
    FILEROOT = '/tmp/'
# The machines of Telecom
else:
    FILEROOT = '/tsi/plato/sons/sd207/'

# Load the cross validation folds
N_FOLDS = 3
train_files, train_scenes, test_fold = np.empty(0, dtype=str), np.empty(0), np.empty(0)
for i in range(N_FOLDS):
    files = pd.read_csv('train%s.txt' % str(i), sep='\s+', header=None)[0].values
    scenes = pd.read_csv('train%s.txt' % str(i), sep='\s+', header=None)[1].values
    print "Fold #%d: %d files from %d sources" % (i+1, len(files), len(np.unique([f.split('_')[0] for f in files])))
    train_files = np.append(train_files, files)
    train_scenes = np.append(train_scenes, scenes)
    test_fold = np.append(test_fold, i*np.ones_like(scenes))

scenes = np.unique(train_scenes)
n_scenes = len(scenes)
labels = pd.factorize(scenes, sort=True)[0]
n_labels = len(labels)
train_labels = pd.factorize(train_scenes, sort=True)[0]
test_files = pd.read_csv('test_files.txt', header=None)[0].values
test_labels = pd.read_csv('meta.txt', header=None)[0].values

print "%d scenes:" % n_scenes, scenes
print "Training set size: %d" % len(train_files)
print "Test set size: %d" % len(test_files)
print "Done in %0.3fs." % (time()-t0)

Loading files...
Fold #1: 290 files from 45 sources
Fold #2: 292 files from 43 sources
Fold #3: 290 files from 45 sources
15 scenes: ['beach' 'bus' 'cafe/restaurant' 'car' 'city_center' 'forest_path'
 'grocery_store' 'home' 'library' 'metro_station' 'office' 'park'
 'residential_area' 'train' 'tram']
Training set size: 872
Test set size: 298
Done in 0.019s.


In [4]:
# Train classifier
print "Doing cross validation..."
t0 = time()

np.random.seed(42)

#feature_type = "mfcc"
n_mfcc = 40
n_fft = 4096
hop_length = 2048
width = 7

clf1 = MLPClassifier(hidden_layer_sizes=(40), alpha=0.01)
y_pred1, y_pred_sum1, y_pred_prod1 = cross_validation(clf1, train_files, train_labels, test_fold, "mfcc")
clf2 = MLPClassifier(hidden_layer_sizes=(40), alpha=0.1)
y_pred2, y_pred_sum2, y_pred_prod2 = cross_validation(clf2, train_files, train_labels, test_fold, "d_mfcc")
print "Done in %0.3fs." % (time()-t0)

Doing cross validation...
Launching fold #1/3, train set size: 582, test set size: 290
fold#1 done in 70.095s, score: 0.614.
Launching fold #2/3, train set size: 580, test set size: 292
fold#2 done in 79.809s, score: 0.668.
Launching fold #3/3, train set size: 582, test set size: 290
fold#3 done in 54.931s, score: 0.562.
Retraining classifier with whole train set...
Done in 110.342s.
Launching fold #1/3, train set size: 582, test set size: 290
fold#1 done in 75.532s, score: 0.628.
Launching fold #2/3, train set size: 580, test set size: 292
fold#2 done in 78.747s, score: 0.630.
Launching fold #3/3, train set size: 582, test set size: 290
fold#3 done in 68.202s, score: 0.559.
Retraining classifier with whole train set...
Done in 127.673s.
Done in 665.335s.


In [5]:
# Print cross validation results
t0 = time()
print '-'*60
print "Prediction result via mfcc features:"
print "Score on validation test (vote by majority): %f" % accuracy_score(train_labels, y_pred1)
print classification_report(train_labels, y_pred1, target_names=scenes)

print "Score on validation test (vote by proba sum): %f" % accuracy_score(train_labels, y_pred_sum1)
print classification_report(train_labels, y_pred_sum1, target_names=scenes)

print "Score on validation test (vote by proba product): %f" % accuracy_score(train_labels, y_pred_prod1)
print classification_report(train_labels, y_pred_prod1, target_names=scenes)
precision1 = precision_score(train_labels, y_pred1, average=None)
print "Precision1: ", precision1

print '-'*60
print "Prediction result via delta mfcc features:"
print "Score on validation test (vote by majority): %f" % accuracy_score(train_labels, y_pred2)
print classification_report(train_labels, y_pred2, target_names=scenes)

print "Score on validation test (vote by proba sum): %f" % accuracy_score(train_labels, y_pred_sum2)
print classification_report(train_labels, y_pred_sum2, target_names=scenes)

print "Score on validation test (vote by proba product): %f" % accuracy_score(train_labels, y_pred_prod2)
print classification_report(train_labels, y_pred_prod2, target_names=scenes)
precision2 = precision_score(train_labels, y_pred2, average=None)
print "Precision2: ", precision2

------------------------------------------------------------
Prediction result via mfcc features:
Score on validation test (vote by majority): 0.614679
                  precision    recall  f1-score   support

           beach       0.72      0.49      0.59        59
             bus       0.59      0.58      0.58        59
 cafe/restaurant       0.74      0.35      0.48        57
             car       0.90      0.64      0.75        59
     city_center       0.88      0.88      0.88        59
     forest_path       0.78      0.95      0.86        60
   grocery_store       0.76      0.69      0.73        59
            home       0.56      0.47      0.51        58
         library       0.62      0.83      0.71        58
   metro_station       0.44      0.59      0.50        56
          office       0.72      0.73      0.73        60
            park       0.49      0.29      0.37        58
residential_area       0.38      0.56      0.45        59
           train       0.70      0.

In [6]:
y_test_pred, y_test_pred_sum= predict_calibration([clf1, clf2], ["mfcc", "d_mfcc"], [precision1, precision2], test_files)

print "Score by maj: %f" % accuracy_score(test_labels, y_test_pred)
print "Score by sum: %f" % accuracy_score(test_labels, y_test_pred_sum)
#print "Score by prod: %f" % accuracy_score(test_labels, y_test_pred_prod)

#np.savetxt('y_test_pred_mfcc_mlp.txt', y_test_pred, fmt='%d')
#np.savetxt('y_test_pred_mfcc_mlp_sum.txt', y_test_pred_sum, fmt='%d')
#np.savetxt('y_test_pred_mfcc_mlp_prod.txt', y_test_pred_prod, fmt='%d')

Score by maj: 0.768456
Score by sum: 0.644295
